# Setup

In [3]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean
import random
import numpy as np
from copy import deepcopy
import itertools
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm


import sys
sys.path.append('/Users/kperks/Documents/ell-connectome/efish_em/efish_em')

# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest
import AnalysisCode as efish 


The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


### Import settings

If you save a copy of settings_dict.json (found in the "under construction" directory of eCREST repo) locally somewhere outside the repo (like in your save_dir), then you can use the following code cell to import. This avoids needing to re-type the save_dir and db_path each time you "git pull" updates from the repo to this notebook.

In [4]:
# path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
# settings_dict = efish.import_settings(path_to_settings_json)

In [71]:
# dirpath = Path(settings_dict['save_dir'])
dirpath = Path('/Users/kperks/Documents/sawtell_lab/EM_data/reconstructions_published')

# dirpath = "/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"

# Get all cells info

In [72]:
nodefiles = efish.get_cell_filepaths(dirpath)

## Cell Types

## from file

In [73]:
df_type = pd.read_csv(dirpath.parent / 'data_processed_published/df_type_auto_typed.csv')

## Base Segments

In [74]:
# Create a base_segments dictionary of all reconstructed cells 

base_segments = {}
for x,f in nodefiles.items():
    # if cell_type[x] in network_types: # if do this, you can't check if the post-syn segments exist as a reconstruction
    cell_data = efish.load_ecrest_celldata(f) #ecrest(settings_dict,filepath = f)#,launch_viewer=False)
    base_segments[cell_data['metadata']['main_seg']['base']] = cell_data['base_segments']
    
    try:
        assert cell_data['metadata']['main_seg']['base'] == x
    except:
        print(x,cell_data['metadata']['main_seg']['base'])

## Cell structure labeling checks

In [117]:
print('need labeling for:')

for x,segs in base_segments.items():
    if (len(segs['unknown']) == len([s for k,v in segs.items() for s in v])) & (cell_type[x] in ['lf','lg']):
        print(f'{x} {cell_type[x]}')

need labeling for:


# Build Graph

In [65]:
network_types = set(['smpl'])#set([v for v in df_type['cell_type'].unique()])#['pe'] #set(['sg1','sg2','smpl','grc','mg1','mg2','lg','lf']) #set([v for v in df_type['cell_type'].unique()])#['smpl']#['tsd']# 

In [82]:
synanno_type = 'post-synaptic'
vx_sizes = [16, 16, 30]

## find edges and set the cell-structure attribute of the edge based on which part of the cell the edge goes to
edge_list = []

with tqdm(total=len(nodefiles.keys())) as pbar:
    for x_pre,f_pre in nodefiles.items():
        pbar.update(1)
        pre = efish.load_ecrest_celldata(f_pre)
        
        # if pre['metadata']['cell-type']['manual'] in network_types: 
        this_type_annotations=[]
        if synanno_type in pre['end_points'].keys():
            for pos, point in enumerate(pre['end_points'][synanno_type]):
                if point[-1] not in ['annotatePoint','annotateBoundingBox','annotateSphere']:
                    point = (point, 'annotatePoint') 
                this_type_annotations.append(point)
            # rewrite end_points for point_type with new formatting
            pre['end_points'][synanno_type] =  this_type_annotations  
        
        # if the node has post-synaptic annotations (the current cell is assumed pre-synaptic)
        # pre = ecrest(settings_dict,filepath = nodefiles[x_pre])#,launch_viewer=False)
        if pre['end_points'][synanno_type] != []:
            # for each synapse
            for syn_ in pre['end_points'][synanno_type]:
                '''assumes that the annotation is a point annotation stored in the list as ([x,y,z,segment_id],'annotatePoint')
                previous ot Jan 25 2024, it was just [x,y,z,segment_id]'''
                syn_ = syn_[0]
                try:
                    post_seg = syn_[3]
                    syn_ = array([int(syn_[i]) for i in range(3)]) # synapses annotations exported as nanometers, so do not need to convert

                    # go through each other nodes
                    for x_post in nodefiles.keys():
                        # if cell_type[x_post] in network_types:
                        post = base_segments[x_post] 
                        for k,v in post.items():
                            for v_ in list(v): #find keys (can be multiple on the same cell) for matching segment ids
                                if post_seg == v_: 
                                    # add edge to the graph between current node and matching node
                                    edge_list.append([x_pre,x_post,k,syn_[0],syn_[1],syn_[2]])
                                        

                except IndexError as msg:
                    cellid = x_pre
                    print(msg, f'for cell {cellid} synapse at {array([int(syn_[i]/vx_sizes[i]) for i in range(3)])} voxels has no segment id')

            else:
                continue


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5025/5025 [46:09<00:00,  1.81it/s]


## Specific cell(S)

In [24]:
edge_list = []
cell_list = ['297083216']
synanno_type = 'post-synaptic'

for x_pre in cell_list:
    pre = ecrest(settings_dict,filepath = nodefiles[x_pre])
    if pre.cell_data['end_points'][synanno_type] != []:
        # for each synapse
        for syn_ in pre.cell_data['end_points'][synanno_type]:
            '''assumes that the annotation is a point annotation stored in the list as ([x,y,z,segment_id],'annotatePoint')
            previous ot Jan 25 2024, it was just [x,y,z,segment_id]'''
            syn_ = syn_[0]
            try:
                post_seg = syn_[3]
                syn_ = array([int(syn_[i]) for i in range(3)]) # synapses annotations exported as nanometers, so do not need to convert

                # go through each other nodes
                for x_post in nodefiles.keys():
                    # if cell_type[x_post] in network_types:
                    post = base_segments[x_post] 
                    for k,v in post.items():
                        for v_ in list(v): #find keys (can be multiple on the same cell) for matching segment ids
                            if post_seg == v_: 
                                # add edge to the graph between current node and matching node
                                
                                edge_list.append([x_pre,x_post,k,syn_[0],syn_[1],syn_[2]])
                                    

            except IndexError as msg:
                cellid = x_pre
                print(msg, f'for cell {cellid} synapse at {array([int(syn_[i]/vx_sizes[i]) for i in range(3)])} voxels has no segment id')

    else:
        continue



# Synapses dataframe

In [83]:
df_syn = pd.DataFrame(edge_list,columns = ['pre','post','structure','x','y','z'])

# for i,r in df_syn.iterrows():
#     df_syn.loc[i,'pre_type']=cell_type[df_syn.loc[i,'pre']]
#     df_syn.loc[i,'post_type']=cell_type[df_syn.loc[i,'post']]

## If want to peak at df_Edges

In [37]:
for i,r in df_syn.iterrows():
    try:
        df_syn.loc[i,'pre_type'] =df_type[df_type['id'].isin([int(r['pre'])])].cell_type.values[0]
        df_syn.loc[i,'post_type']=df_type[df_type['id'].isin([int(r['post'])])].cell_type.values[0]
    except:
        # print(r['pre'],r['post'])
        continue

In [38]:
df_edges = df_syn[['pre','post','pre_type','post_type']].value_counts().reset_index(name='weight') #'structure',

In [29]:
mask = df_edges['pre'].isin(['471122674'])
df_edges#[mask]#[df_edges['post_type'].isin(['mg2','lg'])]

,pre,post,pre_type,post_type,weight
0,297083216,386363638,pe,sg1,4
1,297083216,41843700,pe,mg1,2
2,297083216,130008081,pe,sg1,1
3,297083216,142571993,pe,uk,1
4,297083216,213590737,pe,mg1,1
5,297083216,213669553,pe,mg1,1
6,297083216,297238773,pe,mli,1
7,297083216,310871239,pe,h,1
8,297083216,385388434,pe,sg1,1
9,297083216,387554847,pe,sg1,1


In [40]:
display(df_edges[['pre','post_type']].groupby('post_type')['pre'].count())

post_type
fov    1
lf     1
mg1    1
mg2    3
mli    1
Name: pre, dtype: int64

In [ ]:
for i,r in df_edges.iterrows():
    df_edges.loc[i,'pre_diam']=soma_diam[str(df_edges.loc[i,'pre'])]
    df_edges.loc[i,'post_diam']=soma_diam[str(df_edges.loc[i,'post'])]

df_edges.loc[:,'diam_diff'] = (df_edges['post_diam']-df_edges['pre_diam'])/df_edges['pre_diam']

In [ ]:
# df_edges

focal_cell_id = df_edges['post'].unique()
display(df_syn[df_syn['pre']==focal_cell_id][['post','post_type']].value_counts().reset_index(
    name='weight')['post_type'].value_counts().reset_index(name='ncells'))

# save df_syn

In [84]:
savepath = dirpath.parent / 'data_processed_published'

df_syn.to_csv(savepath / 'df_postsyn.csv')

In [78]:
df_type = pd.read_csv(dirpath.parent / 'data_processed_published/df_type_auto_typed.csv')

no_pre = []
no_post = []
for i,r in df_syn.iterrows():
    # print(r['pre'])
    # df_syn.loc[i,'pre_type'] =df_type[df_type['id'].isin([int(r['pre'])])].cell_type.values[0]
    try:
        df_syn.loc[i,'pre_type'] =df_type[df_type['id'].isin([int(r['pre'])])].cell_type.values[0]
    except:
        no_pre.append(r['pre'])
        continue
    try:
        df_syn.loc[i,'post_type']=df_type[df_type['id'].isin([int(r['post'])])].cell_type.values[0]
    except:
        no_post.append(r['post'])
        continue

In [79]:
no_pre

[]

In [80]:
mask = df_syn['pre_type'].isin(['smpl'])
df_syn[mask]['pre'].unique()

array(['219161561', '220352606', '305051491', '306242528', '389812730',
       '392042360', '45120720', '479153010', '565013265'], dtype=object)

In [64]:
mask = df_type['cell_type'].isin(['smpl'])
df_type[mask]['id'].unique()

array([130053130, 131029691, 131165202, 131211751, 132158911, 132187244,
       132311136, 132962090, 133317438, 133489557, 133517196, 134323277,
       134569151, 134587377, 134679177, 136984946, 215883417, 216977942,
       217088163, 217909620, 218157341, 218169751, 219146053, 219161561,
       220091248, 220290845, 220341622, 220352606, 220354174, 220491948,
       220492782, 220493521, 220602970, 221207249, 221373654, 221574057,
       221592066, 221592085, 221714230, 299482385, 301682826, 301913586,
       302714864, 302950770, 303814400, 305020954, 305051491, 305099311,
       305116146, 305191203, 305407297, 305888407, 305949611, 306242528,
       306274968, 306289985, 306290102, 306338104, 306461050, 306537491,
       307141006, 307157039, 307263751, 307542932, 307559460, 307637261,
       308658207, 308798115, 388576529, 388682967, 388729549, 388778016,
       388807334, 389595457, 389657241, 389812730, 390802240, 390941348,
       390956911, 390990371, 391049801, 391080763, 

In [64]:
df_syn[df_syn['pre'].isin(['463095118','461948885'])]

,pre,post,structure,x,y,z
1043,461948885,299405462,apical dendrite,251584,191232,55740
1044,461948885,213545590,apical dendrite,250896,190960,52830
1045,461948885,214581797,apical dendrite,247792,190224,53430
1046,461948885,132342130,apical dendrite,245328,191232,63120
1047,461948885,386363638,apical dendrite,246512,191504,65460
...,...,...,...,...,...,...
1170,463095118,643848637,apical dendrite,196576,214000,95220
1171,463095118,559041561,multiple,185664,214448,94470
1172,463095118,198648129,unknown,182128,213584,94380
1173,463095118,49314829,unknown,183088,212992,94200


# TODO reconstruction files from synapses

In [11]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = efish.import_settings(path_to_settings_json)

syn_to_find = set()

nodefiles = efish.get_cell_filepaths(dirpath)                


syn_type = 'pre-synaptic'#'spine_inputs' #

vx_sizes = [16, 16, 30]


pfs reconstructed from each type
299496636 mg1 5/22, 
214581797 mg2 1/22, 
301787806 lg, 11/11
393325331 lf, 8/16

In [12]:
# c_id = '301787806'

cells_todo = ['387677355'] #[str(c) for c in ['472361842', '387382792', '387197529', '472051969', '386690280']] #[k for k,v in cell_type.items() if v=='pf'] #['393325331']# 

In [13]:
syn_to_find = set()

for c_id in cells_todo:
    # crest = ecrest(settings_dict,filepath= nodefiles[c_id], launch_viewer=False)
    cell_data = efish.load_ecrest_celldata(nodefiles[c_id])
    for syn_ in cell_data['end_points'][syn_type]: #crest.cell_data['end_points'][syn_type]:
        try:
            syn_to_find.add(syn_[0][3])

        except IndexError as msg:
            cellid = cell_data['metadata']['main_seg']['base']#crest.cell_data['metadata']['main_seg']['base']
            print(msg, f'for cell {cellid} synapse at {array([int(syn_[0][i]/vx_sizes[i]) for i in range(3)])} has no segment id')

In [14]:
len(cell_data['end_points'][syn_type])

99

In [15]:
len(syn_to_find)

70

First, find if any of these post-synaptic segments are already part of reconstructions completed

In [16]:
# crest = ecrest(settings_dict,filepath = nodefiles[cells_todo[0]])

base_segments = efish.get_base_segments_dict(Path(settings_dict['save_dir']))#/'todo/sgx2_394470350_pre')#/ 'todo_presynaptic')# / 'todo_afferent')#'todo_postsynaptic_grc') 

In [17]:
topop=set()
for k,v in base_segments.items():
    if syn_to_find & v != set():
        # print(f'use reconstruction {k}')
        topop = topop.union(syn_to_find & v)
    

len(topop)

67

Adjust "syn_to_find" to eliminate these base segments from the todo list

In [18]:
syn_to_find = syn_to_find.difference(topop)

len(syn_to_find)

3

create crest files for each of the unidentified post-synaptic partners

Save reconstructed_segs as a json to go through manually

In [15]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)


In [19]:
todo_folder_path = Path(settings_dict['save_dir']) / 'todo/sg1_pre'

for segment_id in sorted(list(syn_to_find)):

    cell = ecrest(settings_dict,segment_id = segment_id, launch_viewer=False)
    cell.save_cell_graph(directory_path = todo_folder_path)#'todo_presynaptic/Krista/sgx_394470350')#/Krista/mg_214581797')

Creating base segment graph for cell 309803695 Cell Reconstruction
all base locations for 1 obtained from SQL database
graph created among all_base_segs
1 clusters of connected components. Connecting these clusters with nearest base segments.
weak clusters connected
segments without a location connected
1 clusters in graph (note should/would be only 1 if loaded base ID from agglomo fresh)
Created a CREST instance for NEW Reconstruction of 309803695. No file saved yet -- save manually.
Saved cell 309803695 reconstruction locally at 2025-05-18 17.19.08
Creating base segment graph for cell 392258382 Cell Reconstruction
all base locations for 9 obtained from SQL database
graph created among all_base_segs
1 clusters of connected components. Connecting these clusters with nearest base segments.
weak clusters connected
segments without a location connected
1 clusters in graph (note should/would be only 1 if loaded base ID from agglomo fresh)
Created a CREST instance for NEW Reconstruction of 